In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Patient Risk Prediction").getOrCreate()


In [ ]:

csv_path = "/FileStore/tables/updated_patient_data.csv"
df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.show()


In [ ]:

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["age"], outputCol="features")
data = assembler.transform(df)


In [ ]:

train_data, test_data = data.randomSplit([0.7, 0.3], seed=42)


In [ ]:

from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="risk_label", featuresCol="features")
model = lr.fit(train_data)


In [ ]:

predictions = model.transform(test_data)
predictions.select("patient_id", "age", "risk_label", "prediction", "probability").show()


In [ ]:

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol="risk_label")
accuracy = evaluator.evaluate(predictions)
print(f"Model Accuracy: {accuracy}")


In [ ]:

predictions.select("patient_id", "age", "risk_label", "prediction")     .write.csv("/FileStore/tables/patient_risk_predictions.csv", header=True, mode="overwrite")
